In [1]:
from threading import Thread

class StreamingThread(Thread):
    def __init__(self, ssc):
        Thread.__init__(self)
        self.ssc = ssc
    def run(self):
        ssc.start()
        ssc.awaitTermination()
    def stop(self):
        print('----- Stopping... this may take a few seconds -----')
        self.ssc.stop(stopSparkContext=False, stopGraceFully=True)

In [2]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [3]:
from pyspark.streaming import StreamingContext
from pyspark.sql import Row, SparkSession

In [4]:
def getSparkSessionInstance(sparkConf):
    if ("sparkSessionSingletonInstance" not in globals()):
        globals()["sparkSessionSingletonInstance"] = SparkSession \
            .builder \
            .config(conf=sparkConf) \
            .getOrCreate()
    return globals()["sparkSessionSingletonInstance"]


In [5]:
def predict(title):
    # Replace this with something smarter
    return 'askreddit'

def process(time, rdd):
    if rdd.isEmpty():
        return
    print("========= %s =========" % str(time))
    spark = getSparkSessionInstance(rdd.context.getConf())
    rowRdd = rdd.map(lambda line: Row(subreddit=line.split('\t')[0],
        title=line.split('\t')[1],
        prediction=predict(line.split('\t')[1])))
    df = spark.createDataFrame(rowRdd)
    df.show()

In [6]:
ssc = StreamingContext(sc, 10)

In [7]:
lines = ssc.socketTextStream("seppe.net", 7777)
lines.foreachRDD(process)

In [8]:
ssc_t = StreamingThread(ssc)
ssc_t.start()

========= 2018-04-17 11:35:40 =========
+----------+---------+--------------------+
|prediction|subreddit|               title|
+----------+---------+--------------------+
| askreddit|askreddit|As international ...|
| askreddit|askreddit|What have you giv...|
| askreddit|askreddit|What's a funny en...|
| askreddit|askreddit|What's the weirde...|
| askreddit|askreddit|For people who re...|
| askreddit|askreddit|What is your next...|
| askreddit|askreddit|Reddit, of what f...|
| askreddit|askreddit|Why does a surpri...|
| askreddit|askreddit|Retail workers of...|
| askreddit|askreddit|What is the weird...|
| askreddit|askreddit|Where are you fro...|
| askreddit|askreddit|Who is the cowork...|
| askreddit|askreddit|Why does Bank of ...|
| askreddit|askreddit|How is your month...|
| askreddit|askreddit|[serious] How wil...|
| askreddit|askreddit|Japanese people, ...|
| askreddit|askreddit|How has starting ...|
| askreddit|askreddit|What was your fun...|
| askreddit|askreddit|Redditors, wha

In [9]:
ssc_t.stop()

----- Stopping... this may take a few seconds -----
+----------+---------+--------------------+
|prediction|subreddit|               title|
+----------+---------+--------------------+
| askreddit|   gaming|Where boys became...|
+----------+---------+--------------------+

========= 2018-04-17 11:36:30 =========
+----------+---------+--------------------+
|prediction|subreddit|               title|
+----------+---------+--------------------+
| askreddit|    funny|Oh, no! The Cromb...|
+----------+---------+--------------------+

